## Import amazon review data

In [ ]:
import os
import numpy as np

In [ ]:
import json
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
wd = os.getcwd()

In [ ]:
data_dir = os.path.join(wd,"..","data")

In [ ]:
df = getDF(os.path.join(data_dir, "reviews_Grocery_and_Gourmet_Food.json.gz"))

# Add recall info

## Import list of asins that appear in both datasets

In [ ]:
asins_dir = os.path.join(wd,"..","asins")

In [ ]:
with open(os.path.join(asins_dir, "asin_intersection.txt")) as asin_file:
    asin_intersection= asin_file.read().splitlines()
    asin_file.close()

## Create new dataframe with boolean "recalled" column

In [ ]:
df_recall = pd.DataFrame({"asin": asin_intersection})

In [ ]:
rev_recall = pd.merge(df, df_recall, how = "outer", on = ["asin"], indicator = True)

In [ ]:
rev_recall["recalled"] = rev_recall["_merge"] == "both"

In [ ]:
rev_recall = rev_recall.drop("_merge",1)

## Sort/Separate Data (if needed)

### Sort by recalled status 

In [ ]:
rev_recall_sorted = rev_recall.sort_values(by="recalled", ascending = False)

### Put in two seperate dataframes 

In [ ]:
nonrecall = rev_recall.groupby(["recalled"]).get_group(False)

In [ ]:
recall = rev_recall.groupby(["recalled"]).get_group(True)

# Add metadata

In [ ]:
metadata = getDF(os.path.join(data_dir, "meta_Grocery_and_Gourmet_Food.json.gz"))

In [ ]:
full_df = pd.merge(rev_recall, metadata_col_subset, how = "left", on = ["asin"])